## Project 1 - Surface Type Classification
xz2830  
Xixi Zhou

### Load packages and data

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file
import matplotlib.pyplot as plt
import seaborn as sns
import math
import os
for dirname, _, filenames in os.walk('/data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
x_train=pd.read_csv('../input/career-con-2019/X_train.csv')
y_train=pd.read_csv('../input/career-con-2019/y_train.csv')
x_test=pd.read_csv('../input/career-con-2019/X_test.csv')
sub=pd.read_csv('../input/career-con-2019/sample_submission.csv')
#split X_train
samples=20
time_series=128
start_x = x_train.shape[0] - samples*time_series
X_train_new, X_test_new = x_train.iloc[:start_x], x_train.iloc[start_x:]
# split y_train
start_y = y_train.shape[0] - samples
y_train_new, y_test_new = y_train.iloc[:start_y], y_train.iloc[start_y:]
print('data loaded')

### Data exploration
Check the train set.

In [ ]:
x_train.head()

The measurement number is 128 in each series.  
there are 3810 series.  
Features in train set seems like normal distribution.  
The range of linear_acceleration is large.(max-min).

In [ ]:
x_train.describe()

Train set has no missing data.

In [ ]:
x_train.info()

Check y_train data.

In [ ]:
y_train.head()

X_train have entries:
* Identifiers:row_id,series_id,measurement_number;  
* Orientation:orientation_x,orientation_y,orientation_z,orientation_w;  
* angular velocities:angular_velocity_X, angular_velocity_Y, angular_velocity_z;  
* linear accelerations:linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z.
y_train have entries:
* series_id - is the foreign key reference to X_train.series_id
* group_id
* surface

Check the value of every kind of surface in y_train set.  
There are 9 kinds of surfaces.  
It shows that the number of concrete is largest and the number of hard_tiles is smallest.They are uneven distributed.  

In [ ]:
y_train['surface'].value_counts()

There are also no missing data in y_train.

In [ ]:
y_train.info()

Let's sort the target data by group_id.It shows the the distribution is not even either.

In [ ]:
plt.figure(figsize=(20,6))
sns.countplot(x='group_id',data=y_train,order=y_train.group_id.value_counts().index)
plt.show()

### Feature Engineering
In IMU sensor data,there are four coordinates:X,Y,Z,W.    
In general,we use X,Y,Z to represent coordinates.But Euler Angles have a limitation called "gimbal lock".  
To add features, we transform quaternion coordinates to Euler angles.

In [ ]:
def toEuler(x,y,z,w):
    t0=2.0*(w*x+y*z)
    t1=1.0-2.0*(x*x+y*y)
    Euler_X=math.atan2(t0,t1)
    
    t2=2.0*(w*y-z*x)
    if t2 > +1.0:
        t2=1.0
    if t2<-1.0:
        t2=-1.0
    Euler_Y=math.asin(t2)
    
    t3=(w*z+x*y)*2.0
    t4=1.0-(y**2+z**2)*2.0
    Euler_Z=math.atan2(t3,t4)
    
    return Euler_X,Euler_Y,Euler_Z

In [ ]:
def getEulerFeatures(x,y,z,w):
    xlist,ylist,zlist=[],[],[]
    for i in range(0,len(x)):
        E_x,E_y,E_z=toEuler(x[i],y[i],z[i],w[i])
        xlist.append(E_x)
        ylist.append(E_y)
        zlist.append(E_z)
    return xlist,ylist,zlist

In [ ]:
def getTotal(x,y,z):
    total=(x**2+y**x+z**2)**0.5
    return total

We add features such as Euler angles and r_angl and angl_euler showing below.

In [ ]:
def input_data(data):
    x,y,z,w=data['orientation_X'].tolist(),data['orientation_Y'].tolist(),data['orientation_Z'].tolist(),data['orientation_W'].tolist()
    xlist,ylist,zlist=getEulerFeatures(x,y,z,w)
    data['Euler_X']=xlist
    data['Euler_Y']=ylist
    data['Euler_Z']=zlist
    data['total_angl']=getTotal(data['angular_velocity_X'],data['angular_velocity_Y'],data['angular_velocity_Z'])
    data['total_lin']=getTotal(data['linear_acceleration_X'],data['linear_acceleration_Y'],data['linear_acceleration_Z'])
    
    data['total_xyz']=(data['orientation_X']**2+data['orientation_Y']**2+data['orientation_Z']**2)**0.5
    data['total_euler']=(data['Euler_X']**2+data['Euler_Y']**2+data['Euler_Z']**2)**0.5
    
    data['r_angl']=data['total_lin']/data['total_angl'] 
    data['angl_euler']=data['total_angl']/data['total_euler'] 
    return data   

In [ ]:
X_train_new=input_data(X_train_new)
X_test_new=input_data(X_test_new)
x_train=input_data(x_train)

Let's merge x_train and y_train to find the relationship between surface and features.

In [ ]:
df=pd.merge(x_train,y_train,on='series_id')
df.head()

From figures showing below,the differences between surface is obvious.  
Most features' distribution are normal distribution.  
So in order to improve accuracy,let's add some new features:max,min,range,median,mean,std and etc.

In [ ]:
features=x_train.columns[3:16]
sfs=(y_train['surface'].value_counts()).index
plt.figure(figsize=(15,30))
i=0
for col in features:
    i+=1
    plt.subplot(5,3,i)
    plt.title(col)
    for surface in sfs:
        tp=df[df['surface']==surface]
        sns.kdeplot(tp[col],label=surface)
plt.show()

Add features related to normal distribution.

In [ ]:
def data_cal(data):
    prepd=pd.DataFrame()
    for col in data.columns:
        if col in['row_id','series_id','measurement_number']:
            continue
        prepd[col+'_max']=data.groupby(['series_id'])[col].max()
        prepd[col+'_min']=data.groupby(['series_id'])[col].min()
        prepd[col+'_range']=prepd[col+'_max']-prepd[col+'_min']
        prepd[col+'_mean']=data.groupby(['series_id'])[col].mean()
        prepd[col+'_median']=data.groupby(['series_id'])[col].median()
        prepd[col+'_std']=data.groupby(['series_id'])[col].std()
        
    return prepd

In [ ]:
X_train_new=data_cal(X_train_new)
X_test_new=data_cal(X_test_new)

In [ ]:
X_train_new.head()
#print(X_train_new.shape)

### Label Encoding
After featrue engineering,we need to train model.  
Before that,let's do label encoding first to transform words in 'surface' to numbers which can be identified.

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
y_train_new['surface']=encoder.fit_transform(y_train_new['surface'])

In [ ]:
print(X_train_new.shape)
print(y_train_new['surface'].shape)
print(X_test_new.shape)

We deal with the possible missing data or invalid by using fillna() and replace() functions.

In [ ]:
X_train_new.fillna(0, inplace = True)
X_test_new.fillna(0, inplace = True)

X_train_new.replace(-np.inf, 0, inplace = True)
X_train_new.replace(np.inf, 0, inplace = True)
X_test_new.replace(-np.inf, 0, inplace = True)
X_test_new.replace(np.inf, 0, inplace = True)

### Run Model
In this part,I choose two different models:*GBDT* and *Random Forest*.  
Decision tree is a good model for classification.  
And two model are all combination of decision trees with good quality to predict.But the kernel of two models are different.  
**Random Forest** is a Bagging method.Each tree will be trained independently by using random sample of data.It is hard to overfit compare to GBDT.  
**GBDT ** is a Boosting method.New trees help correct errors of previous trained tree.And it is slower than RF.But it may be overfitting because of noisy.  

Validation Strategy: Straitified KFold

#### GBDT
First model is GBDT by using GradientBoosting Classifier.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

folds=StratifiedKFold(n_splits=3,shuffle=True,random_state=5000)
te=np.zeros((X_test_new.shape[0],9))
tr=np.zeros((X_train_new.shape[0]))
score=0
for i, (tr_index,tar_index) in enumerate(folds.split(X_train_new,y_train_new['surface'])):
    print('fold:',i)
    Model_train=GradientBoostingClassifier(max_depth=10,n_estimators=50)
    Model_train.fit(X_train_new.iloc[tr_index],y_train_new['surface'][tr_index])
    tr[tar_index]=Model_train.predict(X_train_new.iloc[tar_index])   
    score+=Model_train.score(X_train_new.iloc[tar_index],y_train_new['surface'][tar_index])
    print('score',Model_train.score(X_train_new.iloc[tar_index],y_train_new['surface'][tar_index]))
print('avg_score',score/folds.n_splits)

In [ ]:
importance=Model_train.feature_importances_
features=X_train_new.columns
print(type(importance))
plt.figure(figsize=(20,30))
plt.barh(range(len(importance.tolist())),importance.tolist(),tick_label=features.tolist())
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(tr,y_train_new['surface'])

#### RandomForest
Second model is RandomForestClassifier.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, StratifiedKFold
folds=StratifiedKFold(n_splits=5,shuffle=True,random_state=5000)
te=np.zeros((X_test_new.shape[0],9))
tr=np.zeros((X_train_new.shape[0]))
score=0
for i, (tr_index,tar_index) in enumerate(folds.split(X_train_new,y_train_new['surface'])):
    print('fold:',i)
    Model_train=RandomForestClassifier(n_estimators=200,n_jobs=-1)
    Model_train.fit(X_train_new.iloc[tr_index],y_train_new['surface'][tr_index])
    tr[tar_index]=Model_train.predict(X_train_new.iloc[tar_index])
    te+=Model_train.predict_proba(X_test_new)/folds.n_splits
    score+=Model_train.score(X_train_new.iloc[tar_index],y_train_new['surface'][tar_index])
    print('score',Model_train.score(X_train_new.iloc[tar_index],y_train_new['surface'][tar_index]))
print('avg_score',score/folds.n_splits)


In [ ]:
importance=Model_train.feature_importances_
features=X_train_new.columns
print(type(importance))
plt.figure(figsize=(20,30))
plt.barh(range(len(importance.tolist())),importance.tolist(),tick_label=features.tolist())
plt.show()

In [ ]:
print(tr.shape)
print(y_train_new['surface'].shape)
print(te.shape)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(tr,y_train_new['surface'])

### Submission

Obviously,in this problem,RandomForest have higher accuracy and run faster than GBDT.

In [ ]:
y_test_new['surface']=encoder.inverse_transform(te.argmax(axis=1))
y_test_new.to_csv('submission.csv',index=False)
y_test_new.head(50)